In [1]:
#@title Import Dependencies

import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable
import itertools
import pickle
import numpy as np
from numpy import random
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [2]:
from PIL import Image
from typing import TypeVar, Dict

import torch
import torch.nn as nn
from torch.optim.optimizer import Optimizer


Tensor = TypeVar('torch.tensor')

#matplotlib.use('Agg')

from argparse import Namespace
import ast
import os

import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn.functional as F
from typing import TypeVar, Tuple

import copy

In [3]:
!pip install scipy


[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


In [3]:
input_size = 28*28*1 # img_size = (28,28) ---> 28*28=784 in total
batch_size = 200 # the size of input data took for one iteration

In [ ]:
# from torchvision.transforms import v2
# transform_RandomErasing=transforms.Compose([v2.RandomErasing(),
#                               transforms.ToTensor()])

transform=transforms.Compose([transforms.ToTensor()])

train_data = dsets.FashionMNIST(root = './data', train=True, transform = transform, download = True)
test_data = dsets.FashionMNIST(root = './data', train=False, transform = transform, download = True)

100%|██████████| 26421880/26421880 [00:06<00:00, 4188516.11it/s]


Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 133582.18it/s]


Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:02<00:00, 1522064.84it/s]


Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 4165176.89it/s]


Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



NameError: name 'train_gen' is not defined

In [7]:
#@title Loading the data

train_gen = torch.utils.data.DataLoader(dataset = train_data,
                                             batch_size = batch_size,
                                             shuffle = True)

test_gen = torch.utils.data.DataLoader(dataset = test_data,
                                      batch_size = batch_size,
                                      shuffle = False)

no_batches=len(train_gen)
test_no_batches=len(test_gen)
train_data_len=len(train_data)
test_data_len=len(test_data)



In [7]:
#cvmx=torch.zeros([3*64*64,3*64*64],device=device)
images_list=[]
labels_list=[]
no_batches=len(train_gen)
#images_mean=torch.zeros(3,64,64,device=device)
for i ,(images,labels) in enumerate(train_gen):
    images = Variable(images).cuda().detach()
    labels=Variable(labels).cuda().detach()
    # images_mean=images_mean+images.mean(0)
    # im=torch.reshape(images,[images.shape[0],3*64*64])
    # cvmx+=torch.matmul(torch.transpose(im,0,1),im)
    if(i<(len(train_gen))):
        images_list.append(images)
        labels_list.append(labels)



test_images_list=[]
test_labels_list=[]
test_no_batches=len(test_gen)
for i ,(images,labels) in enumerate(test_gen):
    images = Variable(images).cuda().detach()
    labels=Variable(labels).cuda().detach()
    if(i<(len(test_gen))):
        test_images_list.append(images)
        test_labels_list.append(labels)

train_data_len=len(train_data)
test_data_len=len(test_data)



In [8]:
#@title Define model class
import torch.nn as nn
import torch
import torch.nn.functional as F

from typing import TypeVar, Tuple

Tensor = TypeVar('torch.tensor')

class NeuralNet(torch.nn.Module):
    """
    base class for all NN classifiers
    """
    def __init__(self):
        super().__init__()

    def initialize_weights(self):
        for m in self.modules():
            if isinstance(m, torch.nn.Conv2d):
                torch.nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    torch.nn.init.constant_(m.bias, 0)
            elif isinstance(m, torch.nn.BatchNorm2d):
                torch.nn.init.constant_(m.weight, 1)
                torch.nn.init.constant_(m.bias, 0)
            elif isinstance(m, torch.nn.Linear):
                torch.nn.init.normal_(m.weight, 0, 0.01)
                torch.nn.init.constant_(m.bias, 0)


class CNN(NeuralNet):
    """
    CNN for (binary) classification for CelebA, CheXpert
    """

    def __init__(self,
                 in_channels: int = 3,
                 num_classes: int = 2,
                 flattened_size: int = 16384,
                 low_rank: int = 32,
                 batch_norm_mom: float = 1.0):
        """CNN Builder."""
        super(CNN, self).__init__()

        self.conv_layer = nn.Sequential(
            # Conv Layer block 1
            nn.Conv2d(in_channels=in_channels, out_channels=32, kernel_size=3, padding=1),
            nn.Softplus(beta=1.0),
            nn.BatchNorm2d(32,momentum=1.0),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.Softplus(beta=1.0),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.BatchNorm2d(64,momentum=1.0),
            # Conv Layer block 2
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
            nn.Softplus(beta=1.0),
            nn.BatchNorm2d(64,momentum=1.0),
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.Softplus(beta=1.0),
            nn.BatchNorm2d(128,momentum=1.0),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1),
            nn.MaxPool2d(kernel_size=2, stride=2),
            # Conv Layer block 3
            nn.BatchNorm2d(128,momentum=1.0),
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1),
            nn.Softplus(beta=1.0),
            nn.BatchNorm2d(256,momentum=1.0),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
            nn.Softplus(beta=1.0),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.fc_layer = nn.Sequential(
            nn.Flatten(),
            nn.BatchNorm1d(flattened_size,momentum=1.0),
            nn.Linear(flattened_size, low_rank),
            nn.BatchNorm1d(low_rank,momentum=1.0),
            nn.Linear(low_rank,512),            
            nn.Softplus(beta=1.0),
            nn.BatchNorm1d(512,momentum=1.0),
        )

        self.last_layer=nn.Sequential(
            nn.Linear(512, num_classes)
        )

    def forward(self, x: Tensor) -> Tensor:
        """Perform forward."""

        # conv layers
        x = self.conv_layer(x)

        # fc layer
        x = self.fc_layer(x)

        # last layer
        x=self.last_layer(x)

        return x

    def classify(self, x: Tensor) -> Tuple[Tensor, Tensor, Tensor]:
        net_out = self.forward(x)
        acc = F.softmax(net_out, dim=1)
        class_idx = torch.max(net_out, 1)[1]

        return acc, acc[0, class_idx], class_idx
    




class CelebA_CNN(CNN):
    """CNN."""

    def __init__(self,
                 in_channels: int = 3,
                 num_classes: int = 2,
                 #num_classes: int = 1,
                 flattened_size: int = 16384):
        """CNN Builder."""
        super(CelebA_CNN, self).__init__(in_channels=in_channels, num_classes=num_classes,
                                         flattened_size=flattened_size)

class Fashion_MNIST_CNN(CNN):
    """CNN."""

    def __init__(self,
                 in_channels: int = 1,
                 num_classes: int = 10,
                 #flattened_size: int = 6272,
                 flattened_size: int = 2304,
                 low_rank: int = 64,
                 batch_norm_mom: float = 1.0):
        """CNN Builder."""
        super(Fashion_MNIST_CNN, self).__init__(in_channels=in_channels, num_classes=num_classes,
                                         flattened_size=flattened_size)




class CheXpert_CNN(CNN):
    def __init__(self,
                 in_channels: int = 1,
                 num_classes: int = 2,
                 flattened_size: int = 65536):
        """CNN Builder."""
        super(CheXpert_CNN, self).__init__(in_channels=in_channels, num_classes=num_classes,
                                           flattened_size=flattened_size)


class Net(nn.Module):
  def __init__(self, input_size, hidden_size, num_classes):
    super(Net,self).__init__()
    self.fc1 = nn.Linear(input_size, hidden_size)
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(hidden_size, num_classes)

  def forward(self,x):
    out = self.fc1(x)
    out = self.relu(out)
    out = self.fc2(out)
    return out

In [9]:
# net=Fashion_MNIST_CNN()
# n=0
# for par in net.parameters():
#     n+=par.numel()

# n

In [10]:
#@title Define loss-function & optimizer
loss_function = nn.CrossEntropyLoss()


def images_regulariser(): 
    li_reg_loss = 0
    penalized     = [p for name,p in net.named_parameters() if 'bias' not in name]
    not_penalized = [p for name,p in net.named_parameters() if 'bias' in name]
    for p in penalized:
        li_reg_loss += (p**2).sum()*0.5
    #for p in net.parameters():
#        li_reg_loss += (p**2).sum()*0.5
    reg=li_reg_loss/(train_data_len)*l2regconst
    return(reg)


In [11]:
def addnet(net,net2):
    for param1, param2 in zip(net.parameters(), net2.parameters()):
     param1.data += param2.data

def multiplynet(net,a):
   for param1 in net.parameters():
     param1.data *=a

In [ ]:
from dataclasses import dataclass
@dataclass
class hclass:
    h: Tensor
    eta: Tensor
    etam1g: Tensor
    c11: Tensor
    c21: Tensor
    c22: Tensor

def hper2const(h,gam):
    gh=gam.double()*h.double()
    s=torch.sqrt(4*torch.expm1(-gh/2)-torch.expm1(-gh)+gh)
    eta=(torch.exp(-gh/2)).float()
    etam1g=((-torch.expm1(-gh/2))/gam.double()).float()
    c11=(s/gam).float()
    c21=(torch.exp(-gh)*(torch.expm1(gh/2.0))**2/s).float()
    c22=(torch.sqrt(8*torch.expm1(-gh/2)-4*torch.expm1(-gh)-gh*torch.expm1(-gh))/s).float()
    hc=hclass(h=h,eta=eta,etam1g=etam1g,c11=c11,c21=c21,c22=c22)
    return(hc)

def U(x,v,hc):
    xi1=torch.randn(x.size(),device=device)
    xi2=torch.randn(x.size(),device=device)

    xn=x+hc.etam1g*v+hc.c11*xi1
    vn=v*hc.eta+hc.c21*xi1+hc.c22*xi2
    return([xn, vn])

def bounce(x,v,xstar,width):
    vsign=(((x-xstar+width)/(2*width)).floor()% 2)*(-2)+1
    vn=v*vsign
    xn=((x-xstar-width)% (4*width)-2*width).abs()-width+xstar
    # num_outside=((xn-xstar)>width).sum()+((xstar-xn)>width).sum()
    # if(num_outside>0):
    #     print(num_outside)    
    return([xn, vn])

def bouncenet():
    for p,p_star in zip(net.parameters(),net_star.parameters()):
        [p.data, p.v]=bounce(p.data, p.v, p_star.data, 6/torch.sqrt(l2regconst_extra))

def UBU_step(hper2c,images,labels,batch_it):   
    with torch.no_grad():
        for p in list(net.parameters()):

            [p.data,p.v]=U(p.data,p.v,hper2c)

        bouncenet()

    outputsU = net(images)
    loss_likelihood = loss_function(outputsU, labels)  


    grads_reg=[torch.zeros_like(par) for par in net.parameters()]
    net_pars=list(net.parameters())
    with torch.no_grad():
        for it in range(len_params):
            if(list_no_bias[it]):
                grads_reg[it]=net_pars[it].data*l2regconst

    net.zero_grad()
    #loss.backward()
    loss_likelihood.backward()
    with torch.no_grad():
        grads_likelihood=[par.grad*batch_size for par in net.parameters()]
    
        #Normal, no variance reduction
        # for p,p_star in zip(net.parameters(),net_star.parameters()):      
        #     p.v-=hper2c.h*(p.grad*train_data_len+l2regconst_extra*(p.data-p_star.data))

        for p,grad,grad_reg,p_star,grad_star,star_sum_grad in zip(list(net.parameters()),grads_likelihood,grads_reg,list(net_star.parameters()),net_star_grad_list[batch_it],net_star_full_grad):              
            #Using variance reduction
            p.v-=hper2c.h*(grad_reg+star_sum_grad+(grad-grad_star)*no_batches+l2regconst_extra*(p.data-p_star.data))

        for p in list(net.parameters()):
            [p.data,p.v]=U(p.data,p.v,hper2c)

    #bouncenet()
    return(loss_likelihood.data)

def ind_create(batch_it):
    modit=batch_it %(2*no_batches)
    ind=(modit<=(no_batches-1))*modit+(modit>=no_batches)*(2*no_batches-modit-1)
    return ind

In [ ]:
#@title Output arrays
par_runs=64
num_classes=10
num_epochs=60
switch_to_sampling_epoch=20
switch_to_swag_epoch=15

num_swag_epochs=switch_to_sampling_epoch-switch_to_swag_epoch

training_size=no_batches*batch_size
test_size=test_data_len
labels_arr=torch.zeros(training_size)
test_labels_arr=torch.zeros(test_size)
test_prob_arr=torch.zeros([test_size,num_classes,num_epochs,par_runs])

lr = 1e-2
lr_swag=1e-3
h=2.5e-4
l2regconst=torch.tensor(1).detach()
l2regconst_extra=torch.tensor(50).detach()
gam=torch.sqrt(l2regconst_extra)
hper2c=hper2const(torch.tensor(h/2),gam)

for par_it in range(par_runs):
  print("par_it:",par_it,"\n")
  #@title Build the model
  net = Fashion_MNIST_CNN().cuda()
  net.train()
  optimizer = torch.optim.Adam( net.parameters(), lr=lr)
  
  lr_scheduler = torch.optim.lr_scheduler.PolynomialLR(optimizer=optimizer, total_iters=switch_to_swag_epoch,power=1)


  #@title Training the model

  for epoch in range(num_epochs):
    sum_loss=0
    #l2regconst=torch.min(torch.tensor(1+epoch),torch.tensor(switch_to_swag_epoch)).detach()
    net.train()
    if(epoch==(switch_to_swag_epoch-1)):
      net_star2=copy.deepcopy(net)
      net2=copy.deepcopy(net)
      multiplynet(net2,0)
      optimizer=torch.optim.Adam(net.parameters(),lr=lr_swag)

      
    for i in range(no_batches):       
      b=torch.randint(high=no_batches,size=(1,1))
        
      images=images_list[b].detach()
      labels=labels_list[b].detach()
      


      if(epoch<switch_to_sampling_epoch):
        outputs = net(images)    
        loss_likelihood = loss_function(outputs, labels)
        sum_loss=sum_loss+loss_likelihood    
        reg=images_regulariser()
        loss=loss_likelihood+reg
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if(epoch>=(switch_to_swag_epoch)):
          addnet(net2,net)
      else:
        loss_likelihood=UBU_step(hper2c,images,labels,b)
        sum_loss=sum_loss+loss_likelihood




    #if (i+1) % (no_batches) == 0:
    #print("Reg:",reg)
    print('Epoch [%d/%d], Step [%d/%d]' %(epoch+1, num_epochs, i+1, no_batches))
    correct = 0
    total = 0
    





    #for imagest,labelst in eval_gen:
    if epoch==(switch_to_sampling_epoch-1):
      multiplynet(net2,1/(num_swag_epochs*no_batches))
      multiplynet(net,0)
      addnet(net,net2)
      del net2

      net_star=copy.deepcopy(net)
      len_params=len(list(net_star.parameters()))

      #Variance reduction - saving gradients at each batch at x_star
      net_star_grad_list=[]
      net_star_full_grad=[torch.zeros_like(par, device=device) for par in list(net_star.parameters())]
      for i in range(no_batches):
          images=images_list[i]
          labels=labels_list[i]
          outputs=net_star(images)
          loss_likelihood = loss_function(outputs, labels)
          reg=images_regulariser()
          net_star.zero_grad()
          loss_likelihood.backward()
          grads=[par.grad*batch_size for par in list(net_star.parameters())]
          net_star_grad_list.append(grads)
          for g, gi in zip(net_star_full_grad,grads):
            g+=gi          

      len_params=len(list(net_star.parameters()))
      list_no_bias=torch.zeros(len_params)
      pit=0
      for name, p in net_star.named_parameters():
          if 'bias' not in name:
              list_no_bias[pit]=1.0
          pit+=1

      #Initialise velocities
      for par in list(net.parameters()):
        par.v = torch.randn_like(par,device=device)      


    with torch.no_grad():
      net(torch.cat(images_list[0:50],dim=0).detach())        
      net.eval()



      for testit in range(test_no_batches):
        imagest=test_images_list[testit]
        labelst=test_labels_list[testit]
        actual_batch_size=len(imagest)
        test_labels_arr[(testit*batch_size):(testit*batch_size+actual_batch_size)]=labelst.detach().cpu()
        outputt = net(imagest).detach()#.reshape(actual_batch_size).detach()
        _, predictedt = torch.max(outputt,1)

        correct += (predictedt == labelst).sum()
        total += labelst.size(0)

        test_prob_arr[(testit*batch_size):(testit*batch_size+actual_batch_size),:,epoch,par_it]=torch.softmax(outputt,dim=1)
    


    

    #net.train()       
    print('Test accuracy of the model: %.3f %%' %((100*correct)/(total+1)))

    if(epoch<=switch_to_swag_epoch):
      lr_scheduler.step()
    print('Epoch [%d], Average Loss: %0.4f' %(epoch+1, sum_loss/no_batches))
    
  filepath="output_fashion_SG_UBU.pickle"
  with open(filepath,"wb") as file:
    pickle.dump([labels_arr.numpy(),test_labels_arr.numpy(),test_prob_arr.numpy()],file)





par_it: 0 

Epoch [1/60], Step [300/300]
Test accuracy of the model: 86.781 %
Epoch [1], Average Loss: 0.5338
Epoch [2/60], Step [300/300]
Test accuracy of the model: 89.531 %
Epoch [2], Average Loss: 0.3006
Epoch [3/60], Step [300/300]
Test accuracy of the model: 90.411 %
Epoch [3], Average Loss: 0.2498
Epoch [4/60], Step [300/300]
Test accuracy of the model: 90.631 %
Epoch [4], Average Loss: 0.2132
Epoch [5/60], Step [300/300]
Test accuracy of the model: 91.961 %
Epoch [5], Average Loss: 0.1927
Epoch [6/60], Step [300/300]
Test accuracy of the model: 91.261 %
Epoch [6], Average Loss: 0.1747
Epoch [7/60], Step [300/300]
Test accuracy of the model: 91.391 %
Epoch [7], Average Loss: 0.1569
Epoch [8/60], Step [300/300]
Test accuracy of the model: 91.771 %
Epoch [8], Average Loss: 0.1410
Epoch [9/60], Step [300/300]
Test accuracy of the model: 92.231 %
Epoch [9], Average Loss: 0.1355
Epoch [10/60], Step [300/300]
Test accuracy of the model: 92.511 %
Epoch [10], Average Loss: 0.1144
Epoch 

In [16]:
n=0
for par in net.parameters():
    n+=par.numel()
n

1265258

In [17]:
net_star_backup=copy.deepcopy(net_star)

In [9]:
def rps_single(probs, true_label,num_classes):
    outcome=torch.zeros(num_classes)
    outcome[true_label.int()]=1.0
    cum_probs = torch.cumsum(probs,0)
    cum_outcomes = torch.cumsum(outcome,0)
    
    sum_rps = 0
    for i in range(len(outcome)):         
        sum_rps+= (cum_probs[i] - cum_outcomes[i])**2
    
    return sum_rps/(num_classes-1)

def rps_calc(test_probs, true_labels,test_data_len,num_classes):
    rps_vec=torch.zeros(test_data_len)
    for it in range(test_data_len):
        rps_vec[it]=rps_single(test_probs[it,:].reshape(num_classes),true_labels[it],num_classes)
    return rps_vec

def nll_calc(test_probs, true_labels,test_data_len,num_classes):
    res=0
    for it in range(test_data_len):
        res-=torch.max(torch.tensor([torch.log(test_probs[it,true_labels[it].int()]),-100]))
    return res/test_data_len


def adaptive_calibration_error(test_probs,true_labels, test_data_len, num_classes,num_bins=20):
    o=torch.tensor(0.0).detach()
    for k in range(num_classes):
        ind=torch.argsort(test_probs[:,k],stable=True)        
        testprobsk=test_probs[:,k]
        sorted_probs=testprobsk[ind]
        sorted_true_labels=true_labels[ind]

        true_label_is_k = (sorted_true_labels==k).clone().detach().float()
        bins=(torch.tensor(range(test_data_len))/torch.tensor(test_data_len/num_bins)).floor()

        for b in range(num_bins):
            mask = (bins == b)
            if torch.any(mask):
                o += (true_label_is_k[mask] - sorted_probs[mask]).mean().abs()

    return o / (num_bins*num_classes)


def compute_acc_ace_rps_no_bayes(es,par_runs,switch_to_swa_epoch,test_data_len,num_classes,test_prob_arr,test_labels_arr):
    copies=int(par_runs/es)
    ace_arr=torch.zeros(copies)
    rps_arr=torch.zeros(copies)
    nll_arr=torch.zeros(copies)
    accuracy_arr=torch.zeros(copies)

    for it in range(copies):
        test_prob=torch.Tensor(test_prob_arr[:,:,switch_to_swa_epoch-1,it*es:(it+1)*es]).mean(-1).reshape(test_data_len,num_classes)
        ace_arr[it]=adaptive_calibration_error(test_prob,test_labels_arr,test_data_len, num_classes)
        rps_arr[it]=(rps_calc(test_prob, test_labels_arr,test_data_len,num_classes)).mean()
        nll_arr[it]=nll_calc(test_prob, test_labels_arr,test_data_len,num_classes)
        _, predictedt = torch.max(test_prob,1)
        accuracy_arr[it]= (predictedt==test_labels_arr.reshape(1,test_data_len)).sum()/test_data_len
    print("Non-Bayesian, ensemble size:", es)
    print("mean accuracy:",accuracy_arr.mean(),"std:",accuracy_arr.std())
    print("mean ace:",ace_arr.mean(),"std:",ace_arr.std())
    print("mean nll:",nll_arr.mean(),"std:",nll_arr.std())
    print("mean rps:",rps_arr.mean(),"std:",rps_arr.std())
    return [accuracy_arr.mean(),accuracy_arr.std(),ace_arr.mean(),ace_arr.std(),rps_arr.mean(),rps_arr.std(),nll_arr.mean(),nll_arr.std()]

def compute_acc_ace_rps_swa(es,par_runs,switch_to_sampling_epoch,test_data_len,num_classes,test_prob_arr,test_labels_arr):
    copies=int(par_runs/es)
    ace_arr=torch.zeros(copies)
    rps_arr=torch.zeros(copies)
    nll_arr=torch.zeros(copies)
    accuracy_arr=torch.zeros(copies)

    for it in range(copies):
        test_prob=torch.Tensor(test_prob_arr[:,:,switch_to_sampling_epoch-1,it*es:(it+1)*es]).mean(-1).reshape(test_data_len,num_classes)
        ace_arr[it]=adaptive_calibration_error(test_prob,test_labels_arr,test_data_len, num_classes)
        rps_arr[it]=(rps_calc(test_prob, test_labels_arr,test_data_len,num_classes)).mean()
        nll_arr[it]=nll_calc(test_prob, test_labels_arr,test_data_len,num_classes)
        _, predictedt = torch.max(test_prob,1)
        accuracy_arr[it]= (predictedt==test_labels_arr.reshape(1,test_data_len)).sum()/test_data_len
    print("SWA, ensemble size:", es)
    print("mean accuracy:",accuracy_arr.mean(),"std:",accuracy_arr.std())
    print("mean ace:",ace_arr.mean(),"std:",ace_arr.std())
    print("mean nll:",nll_arr.mean(),"std:",nll_arr.std())
    print("mean rps:",rps_arr.mean(),"std:",rps_arr.std())
    return [accuracy_arr.mean(),accuracy_arr.std(),ace_arr.mean(),ace_arr.std(),rps_arr.mean(),rps_arr.std(),nll_arr.mean(),nll_arr.std()]

#Bayesian
def compute_acc_ace_rps_bayes(es,par_runs,switch_to_sampling_epoch,burnin_epochs,num_epochs,test_data_len,num_classes,test_prob_arr,test_labels_arr): 
    copies=int(par_runs/es)
    ace_arr=torch.zeros(copies)
    rps_arr=torch.zeros(copies)
    nll_arr=torch.zeros(copies)
    accuracy_arr=torch.zeros(copies)

    for it in range(copies):
        test_prob=torch.Tensor(test_prob_arr[:,:,(switch_to_sampling_epoch+burnin_epochs):num_epochs,it*es:(it+1)*es]).mean(-1).mean(-1).reshape(test_data_len,num_classes)
        ace_arr[it]=adaptive_calibration_error(test_prob,test_labels_arr,test_data_len, num_classes)
        rps_arr[it]=(rps_calc(test_prob, test_labels_arr,test_data_len,num_classes)).mean()
        nll_arr[it]=nll_calc(test_prob, test_labels_arr,test_data_len,num_classes)
        _, predictedt = torch.max(test_prob,1)
        accuracy_arr[it]= (predictedt==test_labels_arr.reshape(1,test_data_len)).sum()/test_data_len
    print("Bayesian, ensemble size:", es)
    print("mean accuracy:",accuracy_arr.mean(),"std:",accuracy_arr.std())
    print("mean ace:",ace_arr.mean(),"std:",ace_arr.std())
    print("mean nll:",nll_arr.mean(),"std:",nll_arr.std())
    print("mean rps:",rps_arr.mean(),"std:",rps_arr.std())
    return [accuracy_arr.mean(),accuracy_arr.std(),ace_arr.mean(),ace_arr.std(),rps_arr.mean(),rps_arr.std(),nll_arr.mean(),nll_arr.std()]


In [10]:
#Bayesian
burnin_epochs=10
acc,acc_std,ace,ace_std,rps,rps_std,nll,nll_std=(torch.zeros(5),torch.zeros(5),
torch.zeros(5),torch.zeros(5),torch.zeros(5),torch.zeros(5),torch.zeros(5),torch.zeros(5))
for it in range(5):
     [acc[it],acc_std[it],ace[it],ace_std[it],rps[it],rps_std[0],nll[it],nll_std[it]]=\
     compute_acc_ace_rps_bayes(pow(2,it),par_runs,switch_to_sampling_epoch,burnin_epochs,num_epochs,test_data_len,num_classes,test_prob_arr,test_labels_arr)

from scipy.io import savemat
filepath="results_fashion_bayes_SG_UBU.mat"
mdic={"acc":acc.cpu().numpy(),"acc_std":acc_std.cpu().numpy(),"nll": nll.cpu().numpy(),"nll_std":nll_std.cpu().numpy(),\
      "ace": ace.cpu().numpy(),"ace_std":ace_std.cpu().numpy(), "rps":rps.cpu().numpy(),"rps_std":rps_std.cpu().numpy()}
savemat(filepath,mdic)

Bayesian, ensemble size: 1
mean accuracy: tensor(0.9374) std: tensor(0.0013)
mean ace: tensor(0.0030) std: tensor(0.0003)
mean nll: tensor(0.1957) std: tensor(0.0047)
mean rps: tensor(0.0177) std: tensor(0.0003)
Bayesian, ensemble size: 2
mean accuracy: tensor(0.9420) std: tensor(0.0009)
mean ace: tensor(0.0021) std: tensor(0.0002)
mean nll: tensor(0.1734) std: tensor(0.0029)
mean rps: tensor(0.0164) std: tensor(0.0002)
Bayesian, ensemble size: 4
mean accuracy: tensor(0.9441) std: tensor(0.0010)
mean ace: tensor(0.0018) std: tensor(8.6317e-05)
mean nll: tensor(0.1631) std: tensor(0.0017)
mean rps: tensor(0.0157) std: tensor(0.0001)
Bayesian, ensemble size: 8
mean accuracy: tensor(0.9456) std: tensor(0.0008)
mean ace: tensor(0.0018) std: tensor(0.0002)
mean nll: tensor(0.1580) std: tensor(0.0011)
mean rps: tensor(0.0154) std: tensor(9.8979e-05)
Bayesian, ensemble size: 16
mean accuracy: tensor(0.9464) std: tensor(0.0006)
mean ace: tensor(0.0018) std: tensor(0.0002)
mean nll: tensor(0.15